<a href="https://colab.research.google.com/github/IRGarrido/Lunar-Lander-DQN/blob/master/Lunar_Lander_DQN_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Lunar Lander V3 - Deep Q-Learn


**Dependências:**
```
 pip install swig
 pip install gymnasium[box2d]
```

*   OS
*   Random
*   Numpy
*   Torch
*   Gymnasium









## Importação de Dependências

In [25]:
!pip install swig
!pip install gymnasium[box2d]

In [26]:
import os
import random
import numpy as np
import torch
import torch.nn as nn # P/ Rede Neural
import torch.optim as optim # Otimizador
import torch.nn.functional as F # Funções de ativação, perda, convolução, etc (ReLU)
import torch.autograd as autograd # Diferenciação automática (gradiente)
from torch.autograd import Variable
from collections import deque, namedtuple
import gymnasium as gym

## Geração de Ambiente Gymnasium

In [27]:
env = gym.make("LunarLander-v3", continuous=False, gravity=-10.0,
               enable_wind=False, wind_power=15.0, turbulence_power=1.5)
state_shape = env.observation_space.shape
state_size = env.observation_space.shape[0]
number_actions = env.action_space.n # 0 - Do Nothing, 1 - Fire left, 2 - Fire down, 3 - Fire right
observation_space = env.observation_space
print('State shape: ', state_shape)
print('State size: ', state_size)
print('Number of actions: ', number_actions)
print('Observation space: ', observation_space)


State shape:  (8,)
State size:  8
Number of actions:  4
Observation space:  Box([ -2.5        -2.5       -10.        -10.         -6.2831855 -10.
  -0.         -0.       ], [ 2.5        2.5       10.        10.         6.2831855 10.
  1.         1.       ], (8,), float32)


## Estrutura de Memória

In [ ]:
# Memória de aprendizado
class ReplayMemory(object):
    def __init__(self, capacity):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.capacity = capacity
        self.memory = []

    def push(self, event):
        self.memory.append(event)
        # Deleta a memória mais antiga se estiver cheio (capacidade).
        if len(self.memory) > self.capacity:
            del self.memory[0]

    def sample(self, batch_size):
        # Amostragem aleatória do tamanho do batch size
        experiences = random.sample(self.memory, k = batch_size)
        # Converte em tensores pytorch para serem processados pela rede neural
        states = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float().to(self.device)
        actions = torch.from_numpy(np.vstack([e[1] for e in experiences if e is not None])).long().to(self.device)
        rewards = torch.from_numpy(np.vstack([e[2] for e in experiences if e is not None])).float().to(self.device)
        next_states = torch.from_numpy(np.vstack([e[3] for e in experiences if e is not None])).float().to(self.device)
        dones = torch.from_numpy(np.vstack([e[4] for e in experiences if e is not None]).astype(np.uint8)).float().to(self.device)
        return (states, actions, rewards, next_states, dones)

## Definição da Rede Neural

* taxa de aprendizado: 0.0005
* tamanho de batela: 100
* coeficiente gamma: 0.99
* tamanho da memória: 100 mil amostras
* taxa de atualização (parâmetro tau): 0.001
* Número de camadas: 3 (8 - 64 - 4)
* Função de ativação: ReLU - ReLU

In [28]:
# Parâmetros de aprendizado
learning_rate = 5e-4
batch_size =100 #Tamanho da batela
discount_factor = 0.99 # gamma
replay_buffer_size = int(1e5) #Memory
interpolation_parameter = 1e-3

In [29]:
# Rede Neural
class Network(nn.Module):
  def __init__(self,state_size,number_actions,seed=42):
    super(Network, self).__init__()
    self.seed = torch.manual_seed(seed)
    self.fc1 = nn.Linear(state_size, 64)
    self.fc2 = nn.Linear(64, 64) # 3 Camadas de neurônios totalmente conectadas
    self.fc3 = nn.Linear(64, number_actions)


  def forward(self, state):
    x = self.fc1(state)
    x = F.relu(x)
    x = self.fc2(x)
    x = F.relu(x)
    q_values = self.fc3(x)
    return q_values

In [53]:
class DQN(): # Implementando o aprendizado DQN adaptado
  def __init__(self,state_size,number_actions):
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.state_size =state_size
    self.action_size = number_actions
    self.memory = ReplayMemory(replay_buffer_size) # Memória
    self.model = Network(state_size,number_actions).to(self.device) # Rede Neural
    self.optmize = optim.Adam(self.model.parameters()) # Otimizador
    self.target_network = Network(state_size,number_actions).to(self.device)
    self.step=0

  def step(self,state,action,reward,next_state,done):
    self.memory.push((state,action,reward,next_state,done)) # Armazena no buffer
    self.t_step = (self.t_step +1 )%4 # Atualiza os passos
    if self.t_step == 0: # Verifica se é hora de treinar
      if len(self.memory.memory) > batch_size:
        experiences = self.memory.sample(batch_size)
        self.learn(experiences,discount_factor)

  def act(self, state, epsilon = 0.):
    state = torch.from_numpy(state).float().unsqueeze(0).to(self.device) # Tensor
    self.model.eval() # Modo de avaliação
    with torch.no_grad():
      action_values = self.model(state) # Cálculo de Q-values
    self.model.train() # Treinamento
    if random.random() > epsilon:
      return np.argmax(action_values.cpu().data.numpy())
    else:
      return random.choice(np.arange(self.action_size)) # Escolhe uma ação com maior Q-value

  def learn(self, experiences, discount_factor):
    states, next_states, actions, rewards, dones = experiences # Extrai as memorias
    next_q_targets = self.target_qnetwork(next_states).detach().max(1)[0].unsqueeze(1) # Calcula Q-values alvo
    q_targets = rewards + discount_factor * next_q_targets * (1 - dones) # Equação de Bellman (Diferença Temporal)
    q_expected = self.model(states).gather(1, actions)
    loss = F.mse_loss(q_expected, q_targets) # Calcula perdas
    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step()
    self.soft_update(self.model, self.target_qnetwork, interpolation_parameter) # Propaga e atualiza a rede

  def soft_update(self, local_model, target_model, interpolation_parameter):
    for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
      target_param.data.copy_(interpolation_parameter * local_param.data + (1.0 - interpolation_parameter) * target_param.data)


# Aprendizado por Reforço (Deep Q-Learning)

In [54]:
brain = DQN(state_size,number_actions)
reward = 0

In [55]:
print(env.reset())

[ 1.20162964e-04  9.38680655e-01  1.21504024e-02 -1.28852963e-01
 -1.32402871e-04 -2.75225546e-03  0.00000000e+00  0.00000000e+00]


In [49]:
number_episodes = 2000
maximum_number_timesteps_per_episode = 1000
epsilon_starting_value  = 1.0
epsilon_ending_value  = 0.01
epsilon_decay_value  = 0.995
epsilon = epsilon_starting_value
scores_on_100_episodes = deque(maxlen = 100)

for episode in range(1, number_episodes + 1):
  state, _ = env.reset()
  score = 0
  for t in range(maximum_number_timesteps_per_episode):
    action = brain.act(state, epsilon)
    next_state, reward, done, _, _ = env.step(action)
    brain.step(state, action, reward, next_state, done)
    state = next_state
    score += reward
    if done:
      break
  scores_on_100_episodes.append(score)
  epsilon = max(epsilon_ending_value, epsilon_decay_value * epsilon)
  print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)), end = "")
  if episode % 100 == 0:
    print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)))
  if np.mean(scores_on_100_episodes) >= 200.0:
    print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(episode - 100, np.mean(scores_on_100_episodes)))
    torch.save(brain.local_qnetwork.state_dict(), 'checkpoint.pth')
    break

IndexError: invalid index to scalar variable.